# Pachyderm + Determined for the Brain MRI example

### Pachctl install

In [ ]:
!curl -o /tmp/pachctl.tar.gz -L https://github.com/pachyderm/pachyderm/releases/download/v2.3.9/pachctl_2.3.9_linux_amd64.tar.gz && tar -xvf /tmp/pachctl.tar.gz -C /tmp && cp /tmp/pachctl_2.3.9_linux_amd64/pachctl /usr/local/bin 

In [ ]:
!pachctl version --client-only  

### Connect pachctl to the cluster

In [ ]:
!echo '{"pachd_address": "grpc://10.10.162.124:30650"}' | pachctl config set context "determined-seldon-context" --overwrite

In [ ]:
!pachctl config set active-context "determined-seldon-context"

In [ ]:
!pachctl version

In [ ]:
!pachctl list repo

### Create a new repo

In [ ]:
!pachctl create repo brain-mri-new

In [ ]:
!pachctl list repo

### Define the training pipeline, with the new repo as input

In [ ]:
!cat ../../../determined-examples/scratch/brain-mri/pach/brain-mri/00-training-pipeline.json

In [ ]:
!pachctl create pipeline -f ../../../determined-examples/scratch/brain-mri/pach/brain-mri/00-training-pipeline.json

In [ ]:
!pachctl list pipeline

### Check out where the container is coming from (needs to be built)

In [ ]:
!curl 10.10.162.143:9002/v2/_catalog

### Update the pipeline if changes are made to the .json definition file

In [ ]:
!pachctl update pipeline -f ../../../determined-examples/scratch/brain-mri/pach/brain-mri/00-training-pipeline.json --reprocess

### Split the data into 4

In [ ]:
!ls /data/kaggle_3m

In [ ]:
!mkdir -p /data/split/{CS,DU,HT,FG}

In [ ]:
!ls /data/split

In [ ]:
for x in {"CS","DU","FG","HT"}:
    !cp -r /data/kaggle_3m/TCGA_{x}* /data/split/{x}

In [ ]:
!ls /data/split/*

### There is one study left, "EZ". We can use that later

In [ ]:
!ls /data/kaggle_3m/ | grep EZ

### Upload the first commit

In [ ]:
!pachctl put file brain-mri-new@master:/data/CS -f /data/split/CS -r

In [ ]:
!pachctl put file brain-mri-new@master:/data/FG -f /data/split/FG -r

In [ ]:
!pachctl put file brain-mri-new@master:/data/HT -f /data/split/HT -r

In [ ]:
!pachctl put file brain-mri-new@master:/data/DU -f /data/split/DU -r

In [ ]:
!pachctl put file brain-mri-new@master:/dataset-README.md -f dataset-README.md

In [ ]:
!pachctl put file brain-mri-new@master:/data -f /data/split/CS/ -r

In [ ]:
!pachctl list repo

In [ ]:
!pachctl list pipeline

In [ ]:
!pachctl inspect pipeline brain-mri-train

In [ ]:
!pachctl logs --pipeline=brain-mri-train

### Check Determined for the model in the registry

### Test the first candidate model in the notebook

### Upload the second commit

In [ ]:
!pachctl put file brain-mri-new@master:/data -f /data/split/FG -r

### Compare the two models in the notebook

### Upload the third commit

In [ ]:
!pachctl put file brain-mri-new@master:/data -f /data/split/HT -r

### Upload the fourth commit

In [ ]:
!pachctl put file brain-mri-new@master:/data -f /data/split/DU -r

### Check out other branches

In [ ]:
!pachctl put file brain-mri-new@new:/data -f /data/split/CS/TCGA_CS_4942_19970222/ -r